In [1]:
import time

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import errno

from common.camera import *
from common.visualization import *
from common.utils import *
from common.generators import ChunkedGenerator, UnchunkedGenerator
from common.h36m_dataset import Human36mDataset, preprocess_Human36m
from common.visualization import *
from common.model import *
from common.xianhui_dataset import *
import matplotlib
import glob
import plotly
import json
%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)

from scripts.build_model import *
from scripts.train import *
from scripts.eval import *
from scripts.data_preprocessing_cmu_mocap import *

In [2]:
data_root_path = "../wild_data_cmu/output_human/" 

In [3]:
dataset, keypoints = load_and_preprocess_cmu_mocap(data_root_path)

 load 3d data 
 processing 3d data 
 load 2d keypoints 
 processing 2d keypoints 


In [4]:
subjects = sorted(dataset.subjects())
subjects_train = subjects[:23]
subjects_semi = subjects[:23]
subjects_test = subjects[23:]

In [5]:
cameras_valid, poses_valid, poses_valid_2d = fetch(dataset = dataset,
                                                   keypoints = keypoints, 
                                                   subjects = subjects_test, 
                                                   stride = 1)

cameras_train, poses_train, poses_train_2d = fetch(dataset = dataset,
                                                   keypoints = keypoints, 
                                                   subjects = subjects_train, 
                                                   stride = 1)

cameras_semi, _, poses_semi_2d = fetch(dataset = dataset,
                                       keypoints = keypoints,
                                       subjects = subjects_semi,
                                       stride = 1)

## build model

In [6]:
resume = True
filter_widths = [3,3,3,3,3]
model_pos_train, model_pos, model_traj, model_traj_train = build_models(17, 2, 17, filter_widths)

if resume:
    chk_filename = "checkpoints/cmu_epoch_50.bin"
    print('Loading checkpoint', chk_filename)
    checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
    print('This model was trained for {} epochs'.format(checkpoint['epoch']))
    model_pos_train.load_state_dict(checkpoint['model_pos'])
    model_pos.load_state_dict(checkpoint['model_pos'])
    model_traj_train.load_state_dict(checkpoint['model_traj'])
    model_traj.load_state_dict(checkpoint['model_traj'])
    

# load model on gpu
if torch.cuda.is_available():
    model_pos = model_pos.cuda()
    model_pos_train = model_pos_train.cuda()
    model_traj = model_traj.cuda()
    model_traj_train = model_traj_train.cuda()

# copy on multi-gpu
devices = [0, 1,2,3]
model_pos_train = nn.DataParallel(model_pos_train, device_ids=devices)
model_traj_train = nn.DataParallel(model_traj_train, device_ids=devices)

models = {}
models["model_pos_train"] = model_pos_train
models["model_pos"] = model_pos
models["model_traj_train"] = model_traj_train
models["model_traj"] = model_traj

Loading checkpoint checkpoints/cmu_epoch_50.bin
This model was trained for 50 epochs


## build data generator

In [7]:
receptive_field = model_pos.receptive_field()
pad = (receptive_field - 1) // 2
batch_size = 4096 * 2

In [8]:
test_generator = UnchunkedGenerator(cameras_valid, poses_valid, poses_valid_2d, pad=pad, causal_shift=0, augment=False)
train_generator = ChunkedGenerator(batch_size, cameras_train, poses_train, poses_train_2d, 1, pad=pad, causal_shift=0, shuffle=True, augment=False)
train_generator_eval = UnchunkedGenerator(cameras_train, poses_train, poses_train_2d, pad=pad, causal_shift=0, augment=False)
semi_generator = ChunkedGenerator(batch_size, cameras_semi, None, poses_semi_2d, 1, pad=pad, causal_shift=0, shuffle=True, random_seed=4321, augment=False, endless=True)
semi_generator_eval = UnchunkedGenerator(cameras_semi, None, poses_semi_2d, pad=pad, causal_shift=0, augment=False)

# if resume:
#     train_generator.set_random_state(checkpoint['random_state'])
#     semi_generator.set_random_state(checkpoint['random_state_semi'])

generators = {}
generators["test_generator"] = test_generator
generators["train_generator"] = train_generator
generators["train_generator_eval"] = train_generator_eval
generators["semi_generator"] = semi_generator
generators["semi_generator_eval"] = semi_generator_eval

## Optimizer Meta

In [9]:
lr = 0.001
initial_momentum = 0.1
final_momentum = 0.001
lr_decay = 0.99
optimizer = optim.Adam(list(model_pos_train.parameters()) + list(model_traj_train.parameters()),
                               lr=lr, amsgrad=True)
# if resume:
#     optimizer.load_state_dict(checkpoint['optimizer'])
#     lr = checkpoint["lr"]

## Loss

In [10]:
losses = Losses()

## Training Meta

In [11]:
checkpoint_folder = "checkpoints/"
save_freq = 10 
warmup = 0
epochs = 100

if resume:
    start_epoch = checkpoint['epoch']
else:
    start_epoch = 0

In [12]:
lrs = []
momentums = []

In [ ]:
for epoch in range(start_epoch, epochs+1):
    if epoch < warmup:
        skip = True
    else:
        skip = False
        
    models, optimizer, train_losses = train(models, generators, optimizer, pad, skip, False, True, epoch, cmu = True)    
    eval_losses = evaluate(models, generators, pad, cmu = True)
    losses.update(train_losses, eval_losses)    
    print("traj_valid : ", losses.losses_traj_valid[-1])
    print("traj_train_eval : ", losses.losses_traj_train_eval[-1])
    
    # Decay learning rate exponentially
    lr *= lr_decay
    for param_group in optimizer.param_groups:
        param_group['lr'] *= lr_decay

    # Decay BatchNorm momentum
    momentum = initial_momentum * np.exp(-epoch/epochs * np.log(initial_momentum/final_momentum))
    models["model_pos_train"].module.expand_bn.momentum = momentum
    models["model_traj_train"].module.expand_bn.momentum = momentum
    # print(" -- epoch {:3d} -- ".format(i))
    
    lrs.append(lr)
    momentums.append(momentum)
    
    if epoch % save_freq == 0:
        chk_path = os.path.join(checkpoint_folder, 'cmu_epoch_{}.bin'.format(epoch))
        print('Saving checkpoint to', chk_path)
        torch.save({
                'epoch': epoch,
                'lr': lr,
                'random_state': generators["train_generator"].random_state(),
                'optimizer': optimizer.state_dict(),
                'model_pos': states_convert(models["model_pos_train"]),
                'model_traj': states_convert(models["model_traj_train"]),
                'random_state_semi': generators["semi_generator"].random_state(),
            }, chk_path)

 evaluate on unlabel train set : 100%|██████████| 1956/1956 [00:28<00:00, 67.55it/s] 


traj_valid :  0.387113200424957
traj_train_eval :  0.23986167234302702
Saving checkpoint to checkpoints/cmu_epoch_50.bin


 - epoch  51 train :  98%|█████████▊| 440/451 [14:27<00:21,  1.95s/it]

In [ ]:
fig = losses.visualize(start_epoch, epoch)